In [3]:
dataframes_address = "dataframes/"
knowledgebase_address = "knowledgebase/"
model_address = "model/"

medmentions_path = "MedMentions-master/MedMentions-master/full/data/"

#which scispacy basemodel to use
USE_SCIBERT = False
USE_Large =True
USE_BC5CDR=False

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
def read_lines(file_path):
    with open(file_path, 'r') as file:
        for line in file:
            yield line.strip()

In [6]:
dataset = []
file_path = medmentions_path + "corpus_pubtator.txt"
same_pmid=False
abstract=False
for line in read_lines(file_path):
    if len(line.split("|")) == 3 and same_pmid == False:
        pmid, type, text = line.split("|")
        if type == 't':
            same_pmid=True
            abstract = False
            mentions={}
            entities=[]
            cuis=[]
    if len(line.split("|")) == 3 and same_pmid == True:
        abstract = line.split("|")[2]
        text_and_abstract=text + ' ' + abstract
        abstract=True
    if len(line.split('\t')) == 6 and same_pmid == True :
        pmid, start, end, mention, semId, cui = line.split('\t')
        offset=(int(start),int(end))
        links_dict={cui:1.0}
        mentions[offset]=links_dict
        entities.append((int(start),int(end),"ORG"))
        cuis.append(cui)
        abstract=False

    if len(line.split('\t')) == 1 and same_pmid == True and not abstract:
        same_pmid = False
        dataset.append((text_and_abstract,{"links":mentions,"entities":entities},pmid ))

In [8]:
train_dataset = []
test_dataset = []
pmids_train=[]
file_path_train = medmentions_path + "corpus_pubtator_pmids_trng.txt"
for line in read_lines(file_path_train):
    pmids_train.append(line)

pmids_test=[]
file_path_test = medmentions_path + "corpus_pubtator_pmids_test.txt"
for line in read_lines(file_path_test):
    pmids_test.append(line)

for pmid_train in pmids_train:
    train_dataset.extend(data for data in dataset if data[2]==pmid_train)

for pmid_test in pmids_test:
    test_dataset.extend(data for data in dataset if data[2]==pmid_test)

In [10]:
import spacy
if USE_SCIBERT == True and not USE_Large and not USE_BC5CDR:
  nlp=spacy.load("en_core_sci_scibert")
elif USE_Large == True and not USE_SCIBERT and not USE_BC5CDR:
  nlp=spacy.load("en_core_sci_lg")
elif USE_Large == False and not USE_SCIBERT and USE_BC5CDR:
  nlp=spacy.load("en_ner_bc5cdr_md")

/usr/local/lib/python3.10/dist-packages/spacy_transformers/layers/hf_shim.py:137: UserWarning: Error loading saved torch state_dict with strict=True, likely due to differences between 'transformers' versions. Attempting to load with strict=False as a fallback...

If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current 'transformers' and 'spacy-transformers' versions. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


In [11]:
from spacy.training import Example

TRAIN_EXAMPLES = []
if "sentencizer" not in nlp.pipe_names:
    nlp.add_pipe("sentencizer")
sentencizer = nlp.get_pipe("sentencizer")
for text, annotation,_ in train_dataset:

    try:
        example = Example.from_dict(nlp.make_doc(text), annotation)
        example.reference = sentencizer(example.reference)
        TRAIN_EXAMPLES.append(example)
    except:
        continue

In [12]:
from spacy.ml.models import load_kb

entity_linker = nlp.add_pipe("entity_linker", config={"incl_prior": False}, last=True)
entity_linker.initialize(get_examples=lambda: TRAIN_EXAMPLES, kb_loader=load_kb(knowledgebase_address + '/mykb'))

In [13]:
from thinc.api import decaying
from thinc.api import Adam

learn_rates = decaying(0.005, 1e-4)
optimizer = Adam(learn_rate=learn_rates)

In [23]:
from spacy.util import minibatch, compounding
import random

learning_rates = []
losses_list = []

ITER = 10

with nlp.select_pipes(enable=["entity_linker"]):   # train only the entity_linker
    # ITERATIONS NEED TO BE CHANGED BEFORE EXECUTION
    for itn in range(ITER):
        print(itn)
        random.shuffle(TRAIN_EXAMPLES)
        batches = minibatch(TRAIN_EXAMPLES, size=compounding(1.0, 32.0, 1.001))  # increasing batch sizes
        losses = {}
        for batch in batches:
            nlp.update(
                batch,
                drop=0.2,      # prevent overfitting
                losses=losses,
                sgd=optimizer,
            )

            learning_rates.append(optimizer.learn_rate)
            losses_list.append(losses["entity_linker"])

        if itn % 10 == 0:
            print(itn, "Losses", losses)   # print the training loss
print(itn, "Losses", losses)

nlp.to_disk(model_address + "my_nlp")

0
0 Losses {'entity_linker': 4.110993485277548}
0 Losses {'entity_linker': 4.110993485277548}


In [ ]:
nlp_addr = model_address + "my_nlp"
nlp.to_disk(nlp_addr)